In [1]:
import os
import sys
import time
from bson import ObjectId
from datetime import datetime
from gridfs import GridFS
from PIL import Image
from pymongo import MongoClient
sys.path.insert(0, '../..')
from ajna_commons.flask.conf import DATABASE, MONGODB_URI
from ajna_commons.models.bsonimage import BsonImage, BsonImageList
from virasana.models.models import Ocorrencias, Tags


db = MongoClient(host=MONGODB_URI)[DATABASE]
fs = GridFS(db)

if sys.platform == 'win32':
    BASE = 'E:'
else:
    BASE = '/media/ivan/SAMSUNG'

PASTA_BASE = os.path.join(BASE, 'COV - exemplo de imagens')

2019-03-28 23:48:19,747 ajna         INFO     Configuração de log efetuada


Fazendo log de erros e alertas no arquivo  ../../ajna_commons/flask/error.log
Fazendo log de atividade no arquivo  ../../ajna_commons/flask/access.log


In [2]:
def pega_xml(caminho):
    arquivos = os.listdir(caminho)
    xmls = [arquivo for arquivo in arquivos if arquivo[-4:] == '.xml']
    return xmls

def pega_jpg(caminho, xml):
    arquivos = os.listdir(caminho)
    mesmo_nome = [arquivo for arquivo in arquivos if arquivo[-4:] == '.jpg' and arquivo[:-6].find(xml[-4:])]
    menor_nome_jpg = mesmo_nome[0]
    for nome in mesmo_nome[1:]:
        if len(nome) < len(menor_nome_jpg):
            menor_nome_jpg = nome
    return menor_nome_jpg
    

In [3]:
pasta_exportacao = os.path.join(PASTA_BASE, 'Exportação - drogas')
pasta_importacao = os.path.join(PASTA_BASE, 'Importação - falsa declaração')

def monta_lista(pasta):
    lista_diretorios = os.listdir(pasta)
    aprocessar = {}
    for diretorio in lista_diretorios:
        infos = diretorio.split('-')
        ocorrencia = ''
        try:
            numero = infos[0].strip()
            ocorrencia = infos[1].strip()
            caminho_atual = os.path.join(pasta, diretorio)
            xmls = pega_xml(caminho_atual)
            if len(xmls) > 0:
                caminho_completo_xml = os.path.join(pasta, diretorio, xmls[0])
                filename_jpg = pega_jpg(caminho_atual, xmls[0])
                caminho_completo_jpeg = os.path.join(pasta, diretorio, filename_jpg)
                data_jpg = datetime.fromtimestamp(
                    time.mktime(time.localtime(os.path.getmtime(caminho_completo_jpeg)))
                )
                aprocessar[numero] = (caminho_completo_xml, xmls[0],
                                      caminho_completo_jpeg,
                                      filename_jpg, data_jpg, ocorrencia)
        except Exception as err:
            print(err, diretorio)
    return aprocessar


In [4]:
aprocessar_impo = monta_lista(pasta_importacao)
aprocessar_impo

list index out of range Il Pianetta
list index out of range PANIFICADORA RE ALI JUNIOR
list index out of range Peak Automotiva
list index out of range PEB COMERCIO E IMPORTACAO
list index out of range SACMI


{'DFSU2945397': ('/media/ivan/SAMSUNG/COV - exemplo de imagens/Importação - falsa declaração/DFSU2945397 - bagagem com contrafeito/2016102700262100CC.xml',
  '2016102700262100CC.xml',
  '/media/ivan/SAMSUNG/COV - exemplo de imagens/Importação - falsa declaração/DFSU2945397 - bagagem com contrafeito/2016102700262100CCS.jpg',
  '2016102700262100CCS.jpg',
  datetime.datetime(2016, 10, 27, 17, 49, 45),
  'bagagem com contrafeito'),
 'FSCU8231561': ('/media/ivan/SAMSUNG/COV - exemplo de imagens/Importação - falsa declaração/FSCU8231561 - bagagem com armas/2019031300262100HL.xml',
  '2019031300262100HL.xml',
  '/media/ivan/SAMSUNG/COV - exemplo de imagens/Importação - falsa declaração/FSCU8231561 - bagagem com armas/2019031300262100HLS.jpg',
  '2019031300262100HLS.jpg',
  datetime.datetime(2019, 3, 13, 20, 30, 18),
  'bagagem com armas'),
 'HLBU1060204': ('/media/ivan/SAMSUNG/COV - exemplo de imagens/Importação - falsa declaração/HLBU1060204 - imagens de armas/20190314002614003T.xml',
  '201

In [5]:
aprocessar_expo = monta_lista(pasta_exportacao)
aprocessar_expo

{'ACLU2803947': ('/media/ivan/SAMSUNG/COV - exemplo de imagens/Exportação - drogas/ACLU2803947 - 620 kg de cocaína em pó eem farinha/20180713002623002I.xml',
  '20180713002623002I.xml',
  '/media/ivan/SAMSUNG/COV - exemplo de imagens/Exportação - drogas/ACLU2803947 - 620 kg de cocaína em pó eem farinha/20180713002623002IS.jpg',
  '20180713002623002IS.jpg',
  datetime.datetime(2018, 7, 13, 17, 12, 34),
  '620 kg de cocaína em pó eem farinha'),
 'ACLU9635866': ('/media/ivan/SAMSUNG/COV - exemplo de imagens/Exportação - drogas/ACLU9635866 - 500 kg em piso/201608300026230002.xml',
  '201608300026230002.xml',
  '/media/ivan/SAMSUNG/COV - exemplo de imagens/Exportação - drogas/ACLU9635866 - 500 kg em piso/201608300026230002S_stamp.jpg',
  '201608300026230002S_stamp.jpg',
  datetime.datetime(2016, 8, 30, 0, 43, 24),
  '500 kg em piso'),
 'AMFU8822131': ('/media/ivan/SAMSUNG/COV - exemplo de imagens/Exportação - drogas/AMFU8822131 - 212 kg em gelatina/20170420002630000B.xml',
  '20170420002630

In [102]:
def grava_fs(aprocessar, tag):
    for container, linha in aprocessar.items():
        xmlpath = linha[0]
        filename_xml = linha[1]
        with open(xmlpath, 'rb') as xmlfile:
            content_xml = xmlfile.read()
        jpgpath = linha[2]
        filename_jpg = linha[3]
        mdate = linha[4]
        ocorrencia = linha[5]
        with open(jpgpath, 'rb') as jpegfile:
            content_jpg = jpegfile.read()

        metadata_jpg =  \
        {'contentType': 'image/jpeg',
         'UNIDADE': 'ALFSTS',
         'imagem': jpgpath,
         'dataescaneamento': mdate,
         'criacaoarquivo': mdate,
         'modificacaoarquivo': mdate,
         'numeroinformado': container
        }
        metadata_xml =  \
        {'contentType': 'text/xml',
         'UNIDADE': 'ALFSTS',
         'imagem': jpgpath,
         'dataescaneamento': mdate,
         'criacaoarquivo': mdate,
         'modificacaoarquivo': mdate,
         'numeroinformado': container
        }
        _id = fs.put(content_jpg, filename=filename_jpg, metadata=metadata_jpg)
        Ocorrencias(db).add(_id=ObjectId(_id), usuario='ivan', texto=ocorrencia)
        Tags(db).add(_id=ObjectId(_id), usuario='ivan', tag=tag)
        _id_xml = fs.put(content_xml, filename=filename_xml, metadata=metadata_xml)
        print(_id, filename_jpg, metadata_jpg)
        print(_id_xml, filename_xml, metadata_xml)

In [103]:
grava_fs(aprocessar_expo, '1')

5c9cbd261004b37e0164ed1d 20160919002614000MS.jpg {'numeroinformado': 'SUDU5040387', 'imagem': '/media/ivan/SAMSUNG/COV - exemplo de imagens/Exportação - drogas/SUDU5040387 - droga em carne (não pegamos)/20160919002614000MS.jpg', 'contentType': 'image/jpeg', 'criacaoarquivo': datetime.datetime(2016, 9, 19, 14, 4, 36), 'modificacaoarquivo': datetime.datetime(2016, 9, 19, 14, 4, 36), 'UNIDADE': 'ALFSTS', 'dataescaneamento': datetime.datetime(2016, 9, 19, 14, 4, 36)}
5c9cbd261004b37e0164ed20 20160919002614000M.xml {'numeroinformado': 'SUDU5040387', 'imagem': '/media/ivan/SAMSUNG/COV - exemplo de imagens/Exportação - drogas/SUDU5040387 - droga em carne (não pegamos)/20160919002614000MS.jpg', 'contentType': 'text/xml', 'criacaoarquivo': datetime.datetime(2016, 9, 19, 14, 4, 36), 'modificacaoarquivo': datetime.datetime(2016, 9, 19, 14, 4, 36), 'UNIDADE': 'ALFSTS', 'dataescaneamento': datetime.datetime(2016, 9, 19, 14, 4, 36)}
5c9cbd281004b37e0164ed22 20181221002621002ES.jpg {'numeroinformado'

5c9cbd291004b37e0164ed59 20180607002613005NS.jpg {'numeroinformado': 'BMOU4051097', 'imagem': '/media/ivan/SAMSUNG/COV - exemplo de imagens/Exportação - drogas/BMOU4051097 - 670 kg em papel (de Paranaguá)/20180607002613005NS.jpg', 'contentType': 'image/jpeg', 'criacaoarquivo': datetime.datetime(2018, 6, 7, 14, 43, 4), 'modificacaoarquivo': datetime.datetime(2018, 6, 7, 14, 43, 4), 'UNIDADE': 'ALFSTS', 'dataescaneamento': datetime.datetime(2018, 6, 7, 14, 43, 4)}
5c9cbd291004b37e0164ed5c 20180607002613005N.xml {'numeroinformado': 'BMOU4051097', 'imagem': '/media/ivan/SAMSUNG/COV - exemplo de imagens/Exportação - drogas/BMOU4051097 - 670 kg em papel (de Paranaguá)/20180607002613005NS.jpg', 'contentType': 'text/xml', 'criacaoarquivo': datetime.datetime(2018, 6, 7, 14, 43, 4), 'modificacaoarquivo': datetime.datetime(2018, 6, 7, 14, 43, 4), 'UNIDADE': 'ALFSTS', 'dataescaneamento': datetime.datetime(2018, 6, 7, 14, 43, 4)}
5c9cbd291004b37e0164ed5e 2018112800262100FQS.jpg {'numeroinformado': 

5c9cbd2a1004b37e0164ed8b 20170210002621008VS_stamp.jpg {'numeroinformado': 'CAIU2566051', 'imagem': '/media/ivan/SAMSUNG/COV - exemplo de imagens/Exportação - drogas/CAIU2566051 - 385 kg em fubá de milho/20170210002621008VS_stamp.jpg', 'contentType': 'image/jpeg', 'criacaoarquivo': datetime.datetime(2017, 2, 10, 9, 53, 55), 'modificacaoarquivo': datetime.datetime(2017, 2, 10, 9, 53, 55), 'UNIDADE': 'ALFSTS', 'dataescaneamento': datetime.datetime(2017, 2, 10, 9, 53, 55)}
5c9cbd2a1004b37e0164ed8e 20170210002621008V.xml {'numeroinformado': 'CAIU2566051', 'imagem': '/media/ivan/SAMSUNG/COV - exemplo de imagens/Exportação - drogas/CAIU2566051 - 385 kg em fubá de milho/20170210002621008VS_stamp.jpg', 'contentType': 'text/xml', 'criacaoarquivo': datetime.datetime(2017, 2, 10, 9, 53, 55), 'modificacaoarquivo': datetime.datetime(2017, 2, 10, 9, 53, 55), 'UNIDADE': 'ALFSTS', 'dataescaneamento': datetime.datetime(2017, 2, 10, 9, 53, 55)}
5c9cbd2a1004b37e0164ed90 2017080900263000I1S.jpg {'numeroin

5c9cbd2b1004b37e0164edbd 2017112200262100E0S.jpg {'numeroinformado': 'SUDU1506142', 'imagem': '/media/ivan/SAMSUNG/COV - exemplo de imagens/Exportação - drogas/SUDU1506142 - droga na Bélgica/2017112200262100E0S.jpg', 'contentType': 'image/jpeg', 'criacaoarquivo': datetime.datetime(2017, 11, 22, 16, 36, 42), 'modificacaoarquivo': datetime.datetime(2017, 11, 22, 16, 36, 42), 'UNIDADE': 'ALFSTS', 'dataescaneamento': datetime.datetime(2017, 11, 22, 16, 36, 42)}
5c9cbd2b1004b37e0164edc0 2017112200262100E0.xml {'numeroinformado': 'SUDU1506142', 'imagem': '/media/ivan/SAMSUNG/COV - exemplo de imagens/Exportação - drogas/SUDU1506142 - droga na Bélgica/2017112200262100E0S.jpg', 'contentType': 'text/xml', 'criacaoarquivo': datetime.datetime(2017, 11, 22, 16, 36, 42), 'modificacaoarquivo': datetime.datetime(2017, 11, 22, 16, 36, 42), 'UNIDADE': 'ALFSTS', 'dataescaneamento': datetime.datetime(2017, 11, 22, 16, 36, 42)}
5c9cbd2b1004b37e0164edc2 MNBU0023015.jpg {'numeroinformado': 'MNBU0023015', 'im

5c9cbd2d1004b37e0164edea 201903080026300074S.jpg {'numeroinformado': 'MEDU6878737', 'imagem': '/media/ivan/SAMSUNG/COV - exemplo de imagens/Exportação - drogas/MEDU6878737 - 315 kg em transformador/201903080026300074S.jpg', 'contentType': 'image/jpeg', 'criacaoarquivo': datetime.datetime(2019, 3, 8, 6, 55, 33), 'modificacaoarquivo': datetime.datetime(2019, 3, 8, 6, 55, 33), 'UNIDADE': 'ALFSTS', 'dataescaneamento': datetime.datetime(2019, 3, 8, 6, 55, 33)}
5c9cbd2e1004b37e0164edee 201903080026300074.xml {'numeroinformado': 'MEDU6878737', 'imagem': '/media/ivan/SAMSUNG/COV - exemplo de imagens/Exportação - drogas/MEDU6878737 - 315 kg em transformador/201903080026300074S.jpg', 'contentType': 'text/xml', 'criacaoarquivo': datetime.datetime(2019, 3, 8, 6, 55, 33), 'modificacaoarquivo': datetime.datetime(2019, 3, 8, 6, 55, 33), 'UNIDADE': 'ALFSTS', 'dataescaneamento': datetime.datetime(2019, 3, 8, 6, 55, 33)}
5c9cbd2e1004b37e0164edf0 20180622002614007OS.jpg {'numeroinformado': 'MRKU7386732',

...

5c9cbd381004b37e0164efaa 201808240026300021S.jpg {'numeroinformado': 'HASU1413406', 'imagem': '/media/ivan/SAMSUNG/COV - exemplo de imagens/Exportação - drogas/HASU1413406 - 500 kg em couro (pego no exterior)/201808240026300021S.jpg', 'contentType': 'image/jpeg', 'criacaoarquivo': datetime.datetime(2018, 8, 24, 2, 5, 50), 'modificacaoarquivo': datetime.datetime(2018, 8, 24, 2, 5, 50), 'UNIDADE': 'ALFSTS', 'dataescaneamento': datetime.datetime(2018, 8, 24, 2, 5, 50)}
5c9cbd381004b37e0164efad 201808240026300021.xml {'numeroinformado': 'HASU1413406', 'imagem': '/media/ivan/SAMSUNG/COV - exemplo de imagens/Exportação - drogas/HASU1413406 - 500 kg em couro (pego no exterior)/201808240026300021S.jpg', 'contentType': 'text/xml', 'criacaoarquivo': datetime.datetime(2018, 8, 24, 2, 5, 50), 'modificacaoarquivo': datetime.datetime(2018, 8, 24, 2, 5, 50), 'UNIDADE': 'ALFSTS', 'dataescaneamento': datetime.datetime(2018, 8, 24, 2, 5, 50)}
5c9cbd381004b37e0164efaf 2019010800263000B3S.jpg {'numeroinfo

5c9cbd391004b37e0164efd7 201609250026300021S.jpg {'numeroinformado': 'IPXU3672117', 'imagem': '/media/ivan/SAMSUNG/COV - exemplo de imagens/Exportação - drogas/IPXU3672117 - 500 kg em açúcar (pegaram la fora)/201609250026300021S.jpg', 'contentType': 'image/jpeg', 'criacaoarquivo': datetime.datetime(2016, 9, 25, 2, 46, 42), 'modificacaoarquivo': datetime.datetime(2016, 9, 25, 2, 46, 42), 'UNIDADE': 'ALFSTS', 'dataescaneamento': datetime.datetime(2016, 9, 25, 2, 46, 42)}
5c9cbd391004b37e0164efda 201609250026300021.xml {'numeroinformado': 'IPXU3672117', 'imagem': '/media/ivan/SAMSUNG/COV - exemplo de imagens/Exportação - drogas/IPXU3672117 - 500 kg em açúcar (pegaram la fora)/201609250026300021S.jpg', 'contentType': 'text/xml', 'criacaoarquivo': datetime.datetime(2016, 9, 25, 2, 46, 42), 'modificacaoarquivo': datetime.datetime(2016, 9, 25, 2, 46, 42), 'UNIDADE': 'ALFSTS', 'dataescaneamento': datetime.datetime(2016, 9, 25, 2, 46, 42)}
5c9cbd391004b37e0164efdc 20160323002613000BS_stamp.jpg 

5c9cbd3a1004b37e0164f013 20160921002621003XS.jpg {'numeroinformado': 'HASU4974334', 'imagem': '/media/ivan/SAMSUNG/COV - exemplo de imagens/Exportação - drogas/HASU4974334 - droga em gelatina (não pegamos)/20160921002621003XS.jpg', 'contentType': 'image/jpeg', 'criacaoarquivo': datetime.datetime(2016, 9, 21, 3, 9, 36), 'modificacaoarquivo': datetime.datetime(2016, 9, 21, 3, 9, 36), 'UNIDADE': 'ALFSTS', 'dataescaneamento': datetime.datetime(2016, 9, 21, 3, 9, 36)}
5c9cbd3a1004b37e0164f016 20160921002621003X.xml {'numeroinformado': 'HASU4974334', 'imagem': '/media/ivan/SAMSUNG/COV - exemplo de imagens/Exportação - drogas/HASU4974334 - droga em gelatina (não pegamos)/20160921002621003XS.jpg', 'contentType': 'text/xml', 'criacaoarquivo': datetime.datetime(2016, 9, 21, 3, 9, 36), 'modificacaoarquivo': datetime.datetime(2016, 9, 21, 3, 9, 36), 'UNIDADE': 'ALFSTS', 'dataescaneamento': datetime.datetime(2016, 9, 21, 3, 9, 36)}
5c9cbd3a1004b37e0164f018 201608300026230002S_stamp.jpg {'numeroinfo

5c9cbd3b1004b37e0164f040 20190128002630005ES.jpg {'numeroinformado': 'MSCU6374513', 'imagem': '/media/ivan/SAMSUNG/COV - exemplo de imagens/Exportação - drogas/MSCU6374513 - 266 kg em amianto/20190128002630005ES.jpg', 'contentType': 'image/jpeg', 'criacaoarquivo': datetime.datetime(2019, 1, 28, 10, 42, 13), 'modificacaoarquivo': datetime.datetime(2019, 1, 28, 10, 42, 13), 'UNIDADE': 'ALFSTS', 'dataescaneamento': datetime.datetime(2019, 1, 28, 10, 42, 13)}
5c9cbd3b1004b37e0164f043 20190128002630005E.xml {'numeroinformado': 'MSCU6374513', 'imagem': '/media/ivan/SAMSUNG/COV - exemplo de imagens/Exportação - drogas/MSCU6374513 - 266 kg em amianto/20190128002630005ES.jpg', 'contentType': 'text/xml', 'criacaoarquivo': datetime.datetime(2019, 1, 28, 10, 42, 13), 'modificacaoarquivo': datetime.datetime(2019, 1, 28, 10, 42, 13), 'UNIDADE': 'ALFSTS', 'dataescaneamento': datetime.datetime(2019, 1, 28, 10, 42, 13)}
5c9cbd3b1004b37e0164f045 2018102600263000O4S.jpg {'numeroinformado': 'TGHU0826031',

5c9cbd3b1004b37e0164f06e 20161007002613004ES_stamp.jpg {'numeroinformado': 'MSCU7404567', 'imagem': '/media/ivan/SAMSUNG/COV - exemplo de imagens/Exportação - drogas/MSCU7404567 - 283 kg em laranja/20161007002613004ES_stamp.jpg', 'contentType': 'image/jpeg', 'criacaoarquivo': datetime.datetime(2016, 10, 7, 5, 9, 55), 'modificacaoarquivo': datetime.datetime(2016, 10, 7, 5, 9, 55), 'UNIDADE': 'ALFSTS', 'dataescaneamento': datetime.datetime(2016, 10, 7, 5, 9, 55)}
5c9cbd3b1004b37e0164f071 20161007002613004E.xml {'numeroinformado': 'MSCU7404567', 'imagem': '/media/ivan/SAMSUNG/COV - exemplo de imagens/Exportação - drogas/MSCU7404567 - 283 kg em laranja/20161007002613004ES_stamp.jpg', 'contentType': 'text/xml', 'criacaoarquivo': datetime.datetime(2016, 10, 7, 5, 9, 55), 'modificacaoarquivo': datetime.datetime(2016, 10, 7, 5, 9, 55), 'UNIDADE': 'ALFSTS', 'dataescaneamento': datetime.datetime(2016, 10, 7, 5, 9, 55)}
5c9cbd3b1004b37e0164f073 MEDU3999170 A.jpg {'numeroinformado': 'MEDU3999170',

In [104]:
grava_fs(aprocessar_impo, '4')

5c9cbda21004b37e0164f083 201606030026120018S_stamp.jpg {'numeroinformado': 'SZLU9815835', 'imagem': '/media/ivan/SAMSUNG/COV - exemplo de imagens/Importação - falsa declaração/SZLU9815835 - armas/201606030026120018S_stamp.jpg', 'contentType': 'image/jpeg', 'criacaoarquivo': datetime.datetime(2016, 6, 3, 5, 49, 30), 'modificacaoarquivo': datetime.datetime(2016, 6, 3, 5, 49, 30), 'UNIDADE': 'ALFSTS', 'dataescaneamento': datetime.datetime(2016, 6, 3, 5, 49, 30)}
5c9cbda21004b37e0164f086 201606030026120018.xml {'numeroinformado': 'SZLU9815835', 'imagem': '/media/ivan/SAMSUNG/COV - exemplo de imagens/Importação - falsa declaração/SZLU9815835 - armas/201606030026120018S_stamp.jpg', 'contentType': 'text/xml', 'criacaoarquivo': datetime.datetime(2016, 6, 3, 5, 49, 30), 'modificacaoarquivo': datetime.datetime(2016, 6, 3, 5, 49, 30), 'UNIDADE': 'ALFSTS', 'dataescaneamento': datetime.datetime(2016, 6, 3, 5, 49, 30)}
5c9cbda21004b37e0164f088 20151121002612000RS_stamp.jpg {'numeroinformado': 'SUDU7

5c9cbda21004b37e0164f0b7 20190314002614003TS.jpg {'numeroinformado': 'HLBU1060204', 'imagem': '/media/ivan/SAMSUNG/COV - exemplo de imagens/Importação - falsa declaração/HLBU1060204 - imagens de armas/20190314002614003TS.jpg', 'contentType': 'image/jpeg', 'criacaoarquivo': datetime.datetime(2019, 3, 14, 8, 11, 13), 'modificacaoarquivo': datetime.datetime(2019, 3, 14, 8, 11, 13), 'UNIDADE': 'ALFSTS', 'dataescaneamento': datetime.datetime(2019, 3, 14, 8, 11, 13)}
5c9cbda21004b37e0164f0bb 20190314002614003T.xml {'numeroinformado': 'HLBU1060204', 'imagem': '/media/ivan/SAMSUNG/COV - exemplo de imagens/Importação - falsa declaração/HLBU1060204 - imagens de armas/20190314002614003TS.jpg', 'contentType': 'text/xml', 'criacaoarquivo': datetime.datetime(2019, 3, 14, 8, 11, 13), 'modificacaoarquivo': datetime.datetime(2019, 3, 14, 8, 11, 13), 'UNIDADE': 'ALFSTS', 'dataescaneamento': datetime.datetime(2019, 3, 14, 8, 11, 13)}
5c9cbda21004b37e0164f0bd 2017062000263000HFS.jpg {'numeroinformado': 'Z

In [54]:
def grava_bson_image_list(aprocessar, tag):
    bson_image_list = BsonImageList()
    for container, linha in aprocessar.items():
        xmlpath = linha[0]
        filename_xml = linha[1]
        jpgpath = linha[2]
        filename_jpg = linha[3]
        mdate = linha[4]
        ocorrencia = linha[5]

        metadata_jpg =  \
        {'contentType': 'image/jpeg',
         'UNIDADE': 'ALFSTS',
         'imagem': jpgpath,
         'dataescaneamento': mdate,
         'criacaoarquivo': mdate,
         'modificacaoarquivo': mdate,
         'numeroinformado': container,
         'ocorrencias' : [ { 'texto' : ocorrencia, } ],
         'tags' : [ { 'tag' : tag, 'usuario' : 'ivan' } ]
        }
        bson_image_list.addImage(jpgpath, **metadata_jpg)
        metadata_xml =  \
        {'contentType': 'text/xml',
         'UNIDADE': 'ALFSTS',
         'imagem': jpgpath,
         'dataescaneamento': mdate,
         'criacaoarquivo': mdate,
         'modificacaoarquivo': mdate,
         'numeroinformado': container
        }
        bson_image_list.addImage(xmlpath, **metadata_xml)
        
    return bson_image_list

In [55]:
bson_exportacao = grava_bson_image_list(aprocessar_expo, '1')

In [51]:
bson_file_name = os.path.join(pasta_exportacao, 'exportacao_list.bson')
bson_exportacao.tofile(bson_file_name)


In [56]:
bson_importacao = grava_bson_image_list(aprocessar_impo, '4')

In [57]:
bson_file_name = os.path.join(pasta_importacao, 'importacao_list.bson')
bson_importacao.tofile(bson_file_name)
